# 展示如何利用序列轉序列模型，進行問答模型的學習、測試 
## 2019.5.1. Version 1.0 by sekewei@mail.tku.edu.tw

程式目錄架構

/content/drive/My Drive/Colab Notebooks/seq2seq/ 

         chatbot2017.ipynb 模型的訓練及測試
         
         save/model/stc2017_14k/1-1_512/  模型目錄
                  x00_backup_bidir_model.tar
         
         save/training_model/stc2017_14k/  訓練所須字典，批次，語料目錄
                  x00_training_batches_64.tar
                  pairs.tar
                  voc.tar
   
         pytorch_chatbot/
                   __init__.py
                   config.py
                   evaluate.py
                   load.py
                   main.py
                   model.py
                   README.md
                   train.py


# 安裝套件

安裝pytorch環境

In [1]:
! cat /etc/issue
! python --version
! nvidia-smi

Ubuntu 18.04.2 LTS \n \l

Python 3.6.7
Tue Apr 30 12:35:03 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    17W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                     

In [2]:
#!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
!pip3 uninstall torch
#!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl
#!pip3 install torchvision
!pip3 install torch torchvision

Uninstalling torch-1.0.1.post2:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/lib/python3.6/dist-packages/caffe2/*
    /usr/local/lib/python3.6/dist-packages/torch-1.0.1.post2.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.0.1.post2
  Using cached https://files.pythonhosted.org/packages/31/ca/dd2c64f8ab5e7985c4af6e62da933849293906edcdb70dac679c93477733/torch-1.0.1.post2-cp36-cp36m-manylinux1_x86_64.whl


# 載入語料

In [3]:
#coding:utf-8
import os

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


切換工作目錄，列出工作目錄內容

In [4]:
os.chdir('/content/drive/My Drive/Colab Notebooks/seq2seq')
!ls

chatbot2017.ipynb  data  pytorch_chatbot  runtest.ipynb  save


In [0]:
corpusTxt = 'stc2017_14k.txt'
######################################
import codecs

homePath = '/content/drive/My Drive/Colab Notebooks/seq2seq/'
dataPath = 'data/'

filePath = homePath + dataPath + corpusTxt

with codecs.open(filePath, "r", "utf-8") as f:
    corpus_raw = f.readlines()

corpus_raw2 = [line.strip() for line in corpus_raw]

corpus_train = {}
corpus_train['question'] = corpus_raw2[0::2]
corpus_train['answer'] = corpus_raw2[1::2]

## 利用表格檢視訓練資料 

In [12]:
import pandas as pd

pd_train = pd.DataFrame(index=range(len(corpus_train['question'])), 
                        data={'question': corpus_train['question'], 
                              'answer': corpus_train['answer']},
                       columns=['question','answer'])

pd_train

,question,answer
0,就是 太 小 了 ， 哎,所以 我 霸占 了 两 个 位置 ~ 哈 哈哈 ~
1,作弊,我 这 不 叫 作弊 ， 称为 “ 观望 ”
2,花 姐 看 啊 快乐 大本营 。 。 ~,哈哈 ， 花 姐 快 睡 了 哈 。 晚安 ， 爱 你
3,我 呢,哈哈 ， 算 上 你 ， 她 遇上 两 个 了 ！
4,哈哈 你们 还有 骑行 活动 的 啊 。,是 呀 哈哈 ！
5,看来 你 是 一个 真 性情 之 人 ， 敢 爱 敢 恨 的 那种 ！,这 句 话 你 说 就 对了 ， 敢 爱 敢 恨
6,外国 的 天空 很 蓝 ！ 晚上 多 星星 吗 ！ ！,除了 夏威夷 晚上 看到 明亮 的 星星 ， 其他 时间 没有 留意 哦
7,"多 吃 不 长 肉 , 有 这么 神奇 的 东东 吗 ? 37",西湖 银 泰 礼券 吃 多 了 不 长 肉
8,老 南宁 已 走 远,我 的 睡眠 也 是 离 我 越来越 远 了 。 有 失眠 的 么 ？
9,打来 木 。,好 啊 。 我 号码 又 忘 了 。 ！


# 模型訓練

匯入 pytorch_chatbot 套件  
參考: [TRANSLATION WITH A SEQUENCE TO SEQUENCE NETWORK AND ATTENTION](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

In [6]:
import sys
import pytorch_chatbot.main as pcm
import pytorch_chatbot.evaluate as pce

 import train evaluate



## 首次訓練

In [7]:
corpusTxt = 'stc2017_14k.txt'
######################################

#training
#import main
#!python main.py -tr '../Colab Notebooks/data/corpus60w.txt' -la 1 -hi 512 -lr 0.0001 -it 400 -b 64 -p 100 -s 200 
#layer;hidden layer;learning_rate;iteration;batch_size;print/;save/

homePath = '/content/drive/My Drive/Colab Notebooks/seq2seq/'
dataPath = 'data/'

filePath = homePath + dataPath + corpusTxt
cmd_line = "main.py  -tr filePath -la 1 -hi 512 -lr 0.0001 -it 6000 -b 64 -p 100 -s 200"
sys.argv = cmd_line.split()
sys.argv[2] = filePath
args = pcm.parse()
pcm.run(args)

Start loading training data ...
Building encoder and decoder ...


RuntimeError: ignored

## 後續訓練

In [0]:
corpusTxt = 'stc2017_14k.txt'
from_iteration = 3800
###############################

#filePath = '/content/drive/My Drive/Colab Notebooks/nlp_jp/data/corpus1218.txt'
#filePath = '/content/drive/My Drive/Colab Notebooks/nlp_jp/data/' + seg_corpus_name + '.txt'
#modelPath = '/content/drive/My Drive/Colab Notebooks/nlp_jp/save/model/' + seg_corpus_name + '/1-1_512/' + str(iteration) + '_backup_bidir_model.tar'

homePath = '/content/drive/My Drive/Colab Notebooks/seq2seq/'
dataPath = 'data/'

filePath = homePath + dataPath + corpusTxt
modelFile = homePath + 'save/model/' + corpus + '/1-1_512/' + str(from_iteration) + '_backup_bidir_model.tar'
#modelPath = homePath + 'translate2019/' + 'save/model/' + corpusTxt[:-4] + '/1-1_512/' + str(iteration) + '_backup_bidir_model.tar'

cmd_line = "main.py -tr filePath -l modelPath -lr 0.0001 -it 6000 -b 64 -p 100 -s 200"
sys.argv = cmd_line.split()
sys.argv[2] = filePath
sys.argv[4] = modelPath
args = pcm.parse()
pcm.run(args)


# 模型測試

能執行測試的條件有三

1.   有安裝torch套件
2.   有連結網路硬碟及切換工作目錄為  
        /content/drive/My Drive/Colab Notebooks/stc3
3.   有模型及語料字典檔    
        /content/drive/My Drive/Colab Notebooks/stc3/data/corpus2018.txt     
        /content/drive/My Drive/Colab Notebooks/stc3/save/training_data/corpus2018/400_training_batches_64.tar     
        /content/drive/My Drive/Colab Notebooks/stc3/save/training_data/corpus2018/pairs.tar    
        /content/drive/My Drive/Colab Notebooks/stc3/save/training_data/corpus2018/voc.tar  
        /content/drive/My Drive/Colab Notebooks/stc3/save/training_data/corpus2018/1-1_512/400_backup_bidir_model.tar






定義載入模型方法predictLoad()及預測答句方法predict()

In [0]:
import torch
import random
from pytorch_chatbot.train import indexesFromSentence
from pytorch_chatbot.load import loadPrepareData
from pytorch_chatbot.model import nn, EncoderRNN, LuongAttnDecoderRNN

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

#runTest(n_layers, hidden_size, reverse, modelFile, beam_size, inp, corpus):

def predictLoad(pair, corpus, modelFile, n_layers=1, hidden_size=512):
  torch.set_grad_enabled(False)
  voc, pairs = loadPrepareData(corpus)
  embedding = nn.Embedding(voc.n_words, hidden_size)
  encoder = EncoderRNN(voc.n_words, hidden_size, embedding, n_layers)
  attn_model = 'dot'
  decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.n_words, n_layers)

  checkpoint = torch.load(modelFile)
  encoder.load_state_dict(checkpoint['en'])
  decoder.load_state_dict(checkpoint['de'])

  # train mode set to false, effect only on dropout, batchNorm
  encoder.train(False)
  decoder.train(False)

  encoder = encoder.to(device)
  decoder = decoder.to(device)
  
  return encoder, decoder, voc

def predict(encoder, decoder, voc, question, top):
  #question = '今天'
  #question = '现在 刷 朋友 圈 最大 的 快乐 就是 看 代购 们 各种 直播 '
  result_list = []

  if(top==1):
    beam_size = 1
    output_words, _ = pce.evaluate(encoder, decoder, voc, question, beam_size)
    answer = ' '.join(output_words)
    answer = answer.replace('<EOS>','')
    result_list.append(answer)
    #print(output_words)
  else:
    beam_size = top
    output_words_list = pce.evaluate(encoder, decoder, voc, question, beam_size)
    count = 0;
    for output_words, score in output_words_list:
      count = count + 1
      if(count <= top):
        output_sentence = ' '.join(output_words)
        output_sentence = output_sentence.replace('<EOS>','')
        result_list.append(output_sentence)
        #print(" {:.3f} < {}".format(score, output_sentence))
  
  return result_list

def filter(voc, question):
  words = question.split()
  result = []
  for w in words:
    if(w in voc.word2index):
      result.append(w) 
  return ' '.join(result)

設定參數，呼叫方法，載入模型，進行答句預測

In [0]:
corpus = 'stc2017_14k'
iteration = 3000
top = 2
###################################

n_layers = 1
hidden_size = 512

#reverse = false
# /content/drive/My Drive/Colab Notebooks/stc3/save/training_data/corpus2018/voc.tar, pairs.tar, ..training_batches_64.tar

homePath = '/content/drive/My Drive/Colab Notebooks/seq2seq/'
modelFile = homePath + 'save/model/' + corpus + '/1-1_512/' + str(iteration) + '_backup_bidir_model.tar'

en, de, voc = predictLoad(question, corpus, modelFile, n_layers, hidden_size)

Start loading training data ...
  1, question: '为什么 为什么 为什么 你们 都 不 陪 我 看 电影 ! ! [ 怒骂 ]'
       filter: '为什么 为什么 为什么 你们 都 不 陪 我 看 电影 ! ! [ 怒骂 ]'
	'[ 偷 笑 ] [ 偷 笑 ] [ 偷 笑 ] '
	'[ 偷 笑 ] [ 偷 笑 ] [ 偷 笑 ] [ 偷 笑 '
  2, question: '无 语 我 的 qq 被盗 , 求 拯救 ! 讨厌 [ 怒 ]'
       filter: '无 语 我 的 qq 被盗 , 求 拯救 ! 讨厌 [ 怒 ]'
	'[ 吃惊 ] [ 吃惊 ] [ 吃惊 ] [ 吃惊 ] [ 吃惊 ] '
	'[ 吃惊 ] [ 吃惊 ] [ 吃惊 ] [ 吃惊 ] '
  3, question: '中信 银行 , 别 给 老子 再 扯 什么 空头支票 ! ! ! ! !'
       filter: '中信 银行 , 别 给 老子 再 扯 什么 ! ! ! ! !'
	'[ 哈哈 ] [ 哈哈 ] [ 哈哈 ] [ 哈哈 ] [ 哈哈 ] '
	'[ 哈哈 ] [ 哈哈 ] [ 哈哈 ] [ 哈哈 ] '
  4, question: '大 清早 的 烦躁 死 了 ! 没 一点 好 心情 !'
       filter: '大 清早 的 烦躁 死 了 ! 没 一点 好 心情 !'
	'[ 吃惊 ] [ 吃惊 ] [ 吃惊 ] [ 吃惊 ] [ 吃惊 ] '
	'[ 吃惊 ] [ 吃惊 ] [ 吃惊 ] [ 吃惊 ] '
  5, question: '大 早晨 的 就 去 交 论文 , 我 容易 么 ! !'
       filter: '大 早晨 的 就 去 交 论文 , 我 容易 么 ! !'
	'我 也 是 ! ! ! ! ! ! ! ! ! ! ! ! '
	'我 也 是 ! ! ! ! ! ! ! ! ! ! ! '
  6, question: '海南 游 是 破灭 了 [ 怒 ] [ 怒 ] [ 怒 ]'
       filter: '海南 游 是 破灭 了 [ 怒 ] [ 怒 ] [ 怒 ]'
	'咋 啦 ? ? ? ? ? ? ? ? ? ? ? ? ? '
	'咋 

## 批次測試

In [0]:
count=0
answer_list = []
for question in corpus_test:
  count = count + 1
  question2 = filter(voc, question)
  result = predict(en, de, voc, question2, top)
  print("%3d, question: '%s'\n       filter: '%s'" % (count,question,question2))
  answer_list.append(result[0])
  for answer in result:
    print("\t'%s'" % (answer))

以表格呈現模型預測的第1答句

In [0]:
pd_test2 = pd.DataFrame(index=range(len(corpus_test)), 
                        data={'question':corpus_test['question'], 
                              'answer':answer_list},
                       columns=['question','answer'])

pd_test2

,question,questionEmotion,answer
120,"刚 吃 了 早 午餐 , 开始 劳作 啦 [ 兔子 ]",1,[ 吃惊 ] [ 吃惊 ] [ 吃惊 ] [ 吃惊 ] [ 吃惊 ]
121,生日 快乐 ~ 我 对 自己 说 ~,1,生日 快乐 [ 蛋糕 ] [ 蛋糕 ] [ 蛋糕 ] [ 蛋糕 ]
122,五 号 去 厦门 ~ ! 有 谁 有 兴趣 呢 ? 一起 去 啊 。 。 快快 报名 ~ !,1,我 也 想 去 ! ! ! ! ! ! ! ! ! ! !
123,去 做 双皮奶 [ 酷 ],1,我 也 想 吃 [ 馋嘴 ] [ 馋嘴 ] [ 馋嘴 ] [ 馋嘴
124,"做 一个 好 人 的 感觉 , 特别 好 。",1,我 也 是 ! ! ! ! ! ! ! ! ! ! ! !
125,"走过 山路 的 那个 弯 , 一切 都 会 很 好 。 坚信 !",1,我 也 是 ! ! ! ! ! ! ! ! ! ! ! !
126,"听说 下 个 月 公司 有 旅游 , 两 天 一 夜 w",1,[ 吃惊 ] [ 吃惊 ] [ 吃惊 ] [ 吃惊 ] [ 吃惊 ]
127,"喜欢 中心 湖 , 喜欢 草地 [ 鼓掌 ]",1,[ 花心 ] [ 花心 ] [ 花心 ] [ 花心 ] [ 花心 ]
128,越来越 喜欢 顺丰 了 呢 [ 花心 ] [ 花心 ] @ 顺丰 速 运 官 微,1,我 也 想 吃 [ 馋嘴 ] [ 馋嘴 ] [ 馋嘴 ] [ 馋嘴
129,最近 大爱 adele 的 歌 ~ ~ ~,1,[ 吃惊 ] [ 思考 ] [ 思考 ] [ 思考 ]


## 手動測試

In [0]:
question = '终于 有 酒 喝 了 ! 希望 不会 喝 趴下'
========================
question2 = filter(voc, question)
result = predict(en, de, voc, question2, top)
print("%3d, question: '%s'\n       filter: '%s'" % (count,question,question2))
answer_list.append(result[0])
for answer in result:
    print("\t'%s'" % (answer))

In [0]:
corpus = 'stc2017_14k'
iteration = 3000
top = 2
###################################

def hand_test(voc,en,de,top,source):
  source2 = filter(voc, source)
  target = predict(en, de, voc, source2, top)
  print("source: '%s'\nfilter: '%s'" % (source,source2))
  #return target
  for answer in target:
    print("\t'%s'" % (answer))

homePath = '/content/drive/My Drive/Colab Notebooks/seq2seq/'
modelFile = homePath + 'save/model/' + corpus + '/1-1_512/' + str(iteration) + '_backup_bidir_model.tar'

en, de, voc = predictLoad(corpus, modelFile, n_layers=1, hidden_size=512)

while(True):
  source = input("Segmented Question Input: ")
  #seg_source = mecab_parse_space(source)
  hand_test(voc,en,de,top,source)